# Introduction 


COVID-19 is the most challenging crisis that humanity has faced since the second world war. We have seen how it has affected everyone’s life. Almost no country is left untouched from the COVID-19 outbreak. Due to the lack of making the right decision at the right time, many countries have already faced the wrath of this invisible enemy and many more will in the upcoming weeks.In this project, we’ll visualize the COVID-19 hotspot through out world using Folium.

# Contents

1. Importing libraries

2. Access data via an API.

2. Data Preprocessing.

4.  Generate a choropleth map.

5. Generate circular markers.

6. Generate a heat map.

7. Converting Jupyter Notebook into Dashboards.

8. Summary.

# Importing Libraries

In [ ]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

## About Dataset

The data for this notebook is sourced from "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University" and is updated daily.

Official Github repository: https://github.com/CSSEGISandData/COVID-19

The data for this project is accessed through an easy API developed by Kyle Redelinghuys.

Official site: https://covid19api.com/

Here we are accessing Dataset via API

In [ ]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = " "
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data = res.read().decode("UTF-8")

# Data Preprocessing

Here we are converting dataset into JSON format.

In [ ]:
covid1=json.loads(data)

In [ ]:
pd.json_normalize(covid1["Countries"],sep=",")

Here we are converting to pandas dataframe.

In [ ]:
df = pd.DataFrame(covid1["Countries"])
df

Here we are droping unnecessary columns

In [ ]:
covid2 = df.drop(columns=["CountryCode","Slug","Date","Premium"],axis=1)
covid2

# Generate a choropleth map

Firstly here we are generating base map

In [ ]:
m=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
m

Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json


Here we are obtaining Geodata from geodata url

In [ ]:
url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
country_shapes=f"{url}/world-countries.json"

Here we are Generate Choropleth Map Layer which is showing Total Confirmed Cases through out world.

In [ ]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name="Covid-19",
    data=covid2,
    columns=["Country","TotalConfirmed"],
    key_on="feature.properties.name",
    fill_color="OrRd",
    nan_fill_color="black",
    legend_name="Total Confirmed Covid Cases",
).add_to(m)
m

Here we are generating circular markers on map to get clear view of data.

In [ ]:
covid2.update(covid2["TotalConfirmed"].map("TotalConfirmed:{}".format))
covid2.update(covid2["TotalRecovered"].map("TotalRecovered:{}".format))

Here we are getting corrdinates of covid map from the link
coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv.

In [ ]:
coordinates=pd.read_csv("https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv")
coordinates

Here we are merging covid2 data.

In [ ]:
covid_final=pd.merge(covid2,coordinates,on="Country")

# Generating Circular Markers

Here we can make popup box on map which will be showing Total Number Of Confirmed case in each country.

In [ ]:
def plotDot(point):
  folium.CircleMarker(location=[point.latitude,point.longitude],
                      radius=5,
                      weight=2,
                      tooltip="click me!",
                      popup=folium.Popup(point.TotalConfirmed,point.TotalConfirmed,point.TotalRecovered),
                      fill_color="#000000").add_to(m)

In [ ]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

# Generating Heat Map

Here we are generating heat map so we will start from generating Base map.

In [ ]:
m1=folium.Map(tiles="StamenToner",min_zoom=2)
m1

Here we are generating Heat Map Layer.

In [ ]:
deaths = covid_final["TotalDeaths"].astype(float)
lat = covid_final["latitude"].astype(float)
lon = covid_final['longitude'].astype(float)
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

# Generating Circular Markers

Here we are generating circular marker.|

Here we can make popup box on map which will be showing name of each country.

In [ ]:
def plotDot(point):
  folium.CircleMarker(location=[point.latitude,point.longitude],
                      radius=5,
                      weight=2,
                      tooltip = "click me!",
                      popup=folium.Popup(point.Country,point.TotalDeaths),
                      fill_color="#000000").add_to(m1)

In [ ]:
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1

# Converting Jupyter Notebook into Dashboards.

You can also covert Jupyter Notebook into Dashboards.


To create a dashboard follow these steps:

1) Make a copy of the notebook that you receive after completing this project. Rename it 'covidmap'. (Or any other short name)

2) Remove all Markdown cells.

3) The only output that should be visible are the final choropleth map with markers and the final heatmap with markers. To prevent other outputs from being displayed, just add a semicolon at the end of the code and rerun the kernel.

4) To add a heading above the maps to indicate what they represent, just add a markdown cell above the cell containing the map output.

5)  Save your notebook. Ensure that it is in the same directory as your jupyter lab installation.

6) Open the terminal and type in:

  jupyter nbconvert covidmap1.ipynb --no-input --no-prompt
Note: If you named the copy of the notebook something else, replace that name where I have used'covidmap.ipynb'

7) You will now have a file named 'covidmap.html'




Thanks for reading. I hope you like my World Map Geovisualization found it to be helpful. If you have any questions or suggestions, feel free to write them down in the comment section.